<a href="https://colab.research.google.com/github/mankings/RI/blob/main/assign_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers ranx accelerate
!git clone https://github.com/ua-deti-information-retrieval/Neural-IR-hands-on.git



fatal: destination path 'Neural-IR-hands-on' already exists and is not an empty directory.


In [2]:
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/miguelamatos_ua_pt/Ed_ehOIfi2NNiAF8vxUTRfABr_tCyQXDP7XOIttk4FxMeA?e=TMfyOU&download=1" -O collection_with_texts.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/miguelamatos_ua_pt/EeGvLK--QrRIlP1HdC6I_twBebkRx2IHPmZbLd16hsrPxA?e=OqYEkG&download=1" -O dev_set_nq_gs.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/miguelamatos_ua_pt/ETSankJ3TTNHiMYlDtDZyicBCM8F8iDrkzRA2O5dbyU7LQ?e=5G9uJz&download=1" -O train_set_nq_neg.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/miguelamatos_ua_pt/EWclk_Z3uPhGk2cKSh6KDJYBQPDv-YCsFPSlmBWIO3E4rA?e=0bLWqO&download=1" -O train_set_nq_pos.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/miguelamatos_ua_pt/EdiuKwV4UmZGgOLWctyJawgB1DaWKO5Ejj5DrSPUnO3oRQ?e=h9ppty&download=1" -O dev_set_nq_bm25.jsonl

--2024-02-03 20:03:43--  https://uapt33090-my.sharepoint.com/:u:/g/personal/miguelamatos_ua_pt/Ed_ehOIfi2NNiAF8vxUTRfABr_tCyQXDP7XOIttk4FxMeA?e=TMfyOU&download=1
Resolving uapt33090-my.sharepoint.com (uapt33090-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to uapt33090-my.sharepoint.com (uapt33090-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/miguelamatos_ua_pt/Documents/ri/collection_with_texts.jsonl?ga=1 [following]
--2024-02-03 20:03:44--  https://uapt33090-my.sharepoint.com/personal/miguelamatos_ua_pt/Documents/ri/collection_with_texts.jsonl?ga=1
Reusing existing connection to uapt33090-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 2172688290 (2.0G) [application/octet-stream]
Saving to: ‘collection_with_texts.jsonl’

collection_with_tex 100%[===================>]   2.02G  55.0MB/s    in 45s     

2024-02-03 20:04:29 (45.6 MB/s) - ‘collec

# Starting here

In [3]:
!cd Neural-IR-hands-on && git pull

!cp Neural-IR-hands-on/trainer/* .

Already up to date.


In [4]:
from data import get_qrels, InferenceRankingIterator,  BioASQPointwiseIterator, InferenceDataset, create_training_dataset
from sampler import BasicSampler
from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.model_max_length = 512

# torch.utils.data.Dataset (Iterable)
train_ds = create_training_dataset("train_set_nq_pos.jsonl", # "body"/"question" q_id:[doc_id] 60K
                                   "train_set_nq_neg.jsonl", # q_id:[{"id":doc_id, "score": doc_id}]
                                   "collection_with_texts.jsonl", # doc_id: text (title + " " + abstract)
                                    tokenizer=tokenizer,
                                    iterator_class=BioASQPointwiseIterator[BasicSampler],
                                    #max_questions=500, # debug
                                            )
# torch.utils.data.Dataset (Iterable)
dev_ds = InferenceDataset("dev_set_nq_bm25.jsonl", #q_id:[doc_id ->>] BM25 top-1000 top-100 # 100 question -> 1000 docs
                          train_ds.collection,
                          tokenizer,
                          #max_questions=10, # debug
                          at=100, #max docs
                          gs_path="dev_set_nq_gs.jsonl", # q_id: [doc_id]
                          iterator_class=InferenceRankingIterator)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
from transformers import AutoModelForSequenceClassification

id2label = {0: "IRRELEVANT", 1: "RELEVANT"}
label2id = {"IRRELEVANT": 0, "RELEVANT": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
).to("cuda")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
_iter = iter(train_ds)

print(next(_iter))
print(next(_iter))



{'input_ids': [101, 14494, 1999, 2029, 4962, 1998, 2029, 5250, 2024, 3378, 2007, 5658, 27949, 8715, 1029, 102, 14494, 1999, 6714, 2243, 2629, 1010, 17181, 1037, 14262, 3170, 4013, 27058, 3366, 24054, 1010, 3426, 5658, 27949, 8715, 1012, 2057, 6235, 2182, 5408, 2367, 14494, 1999, 6714, 2243, 2629, 1010, 17181, 1996, 14262, 3170, 4013, 27058, 3366, 24054, 3393, 22462, 1010, 1999, 2410, 2945, 2007, 5658, 27949, 8715, 1006, 24978, 1010, 2771, 2213, 17788, 26187, 8889, 1007, 1012, 2087, 1997, 2122, 14494, 16014, 21371, 18287, 19429, 5644, 1012, 2122, 3463, 26056, 1037, 4187, 2535, 1997, 6714, 2243, 2629, 1999, 4958, 18688, 9067, 8803, 3853, 1998, 15403, 1010, 1998, 6592, 1037, 2047, 12732, 2005, 2152, 20194, 1045, 3351, 3798, 1998, 10234, 2594, 24491, 2015, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(num_train_epochs=14,
                                  learning_rate=2e-5, # AdamW
                                  weight_decay=0.01,
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=32,
                                  evaluation_strategy= "epoch",
                                  dataloader_pin_memory=True,
                                  output_dir="trained_model",
                                  logging_strategy="steps",
                                  logging_first_step=True,
                                  logging_steps=100,
                                  save_strategy="epoch",
                                  save_total_limit=2,
                                  seed=42)

In [8]:
from ranker_trainer import RankerTrainer
from transformers import DataCollatorWithPadding
from collator import RankingCollator
from metrics import RanxMetrics

import torch
trainer = RankerTrainer(
      model=model,
      args=training_args,
      train_dataset=train_ds,
      eval_dataset=dev_ds, # validation set
      tokenizer=tokenizer,
      data_collator=DataCollatorWithPadding(tokenizer=tokenizer), # apply to train
      eval_data_collator=RankingCollator(tokenizer=tokenizer), # apply to val
      preprocess_logits_for_metrics=lambda logits, labels: torch.nn.functional.softmax(logits, dim=-1)[:,1], # prob for class 1 (relevant class)
      compute_metrics=RanxMetrics(dev_ds.get_qrels()),
  )

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [9]:
trainer.train()

Token indices sequence length is longer than the specified maximum sequence length for this model (677 > 512). Running this sequence through the model will result in indexing errors
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Ndcg@100,Ndcg@10,Recall@10,Map@10
1,0.020900,No log,0.203088,0.203717,0.173239,0.145294
2,0.010600,No log,0.200035,0.199633,0.171492,0.140978
3,0.009500,No log,0.206496,0.210107,0.180367,0.149738
4,0.000100,No log,0.207223,0.211783,0.181911,0.150240
5,0.004000,No log,0.211783,0.216640,0.182327,0.155383


/usr/local/lib/python3.10/dist-packages/ranx/metrics/ndcg.py:72: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _ndcg(qrels[i], run[i], k, rel_lvl, jarvelin)


TrainOutput(global_step=1780, training_loss=0.02083992798383651, metrics={'train_runtime': 4147.3843, 'train_samples_per_second': 3.431, 'train_steps_per_second': 0.429, 'total_flos': 3343506347093520.0, 'train_loss': 0.02083992798383651, 'epoch': 5.0})

In [22]:
# Compress the folder
!zip -r trained_model.zip trained_model

# Download the ZIP file
from google.colab import files
files.download('trained_model.zip')

  adding: trained_model/ (stored 0%)
  adding: trained_model/checkpoint-1780/ (stored 0%)
  adding: trained_model/checkpoint-1780/vocab.txt (deflated 53%)
  adding: trained_model/checkpoint-1780/model.safetensors (deflated 7%)
  adding: trained_model/checkpoint-1780/config.json (deflated 50%)
  adding: trained_model/checkpoint-1780/tokenizer_config.json (deflated 76%)
  adding: trained_model/checkpoint-1780/rng_state.pth (deflated 25%)
  adding: trained_model/checkpoint-1780/tokenizer.json (deflated 71%)
  adding: trained_model/checkpoint-1780/trainer_state.json (deflated 76%)
  adding: trained_model/checkpoint-1780/training_args.bin (deflated 50%)
  adding: trained_model/checkpoint-1780/scheduler.pt (deflated 56%)
  adding: trained_model/checkpoint-1780/special_tokens_map.json (deflated 42%)
  adding: trained_model/checkpoint-1780/optimizer.pt (deflated 16%)
  adding: trained_model/.ipynb_checkpoints/ (stored 0%)
  adding: trained_model/runs/ (stored 0%)
  adding: trained_model/runs/F

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Use a trained model to run inference!

## TODO reboot the machine

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = "trained_model/checkpoint-1780"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.model_max_length = 512

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).to("cuda")

In [15]:
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/miguelamatos_ua_pt/EXMS_I8we6ZPuHzVHQmurMEBNd9MG4UKFOWSKI503E7eAA?e=RYC9i2&download=1" -O BM25_E9B1.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/miguelamatos_ua_pt/EXS9EPpsgQREmvJtpRK81F4Be1JIZJP7KHch76u9ER4wEQ?e=IRL9JI&download=1" -O BM25_E9B2.jsonl

--2024-02-03 21:53:55--  https://uapt33090-my.sharepoint.com/:u:/g/personal/miguelamatos_ua_pt/EXMS_I8we6ZPuHzVHQmurMEBNd9MG4UKFOWSKI503E7eAA?e=RYC9i2&download=1
Resolving uapt33090-my.sharepoint.com (uapt33090-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to uapt33090-my.sharepoint.com (uapt33090-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/miguelamatos_ua_pt/Documents/ri/BM25_E9B1.jsonl?ga=1 [following]
--2024-02-03 21:53:56--  https://uapt33090-my.sharepoint.com/personal/miguelamatos_ua_pt/Documents/ri/BM25_E9B1.jsonl?ga=1
Reusing existing connection to uapt33090-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 496112 (484K) [application/octet-stream]
Saving to: ‘BM25_E9B1.jsonl’

BM25_E9B1.jsonl     100%[===================>] 484.48K   630KB/s    in 0.8s    

2024-02-03 21:53:57 (630 KB/s) - ‘BM25_E9B1.jsonl’ saved [496112/496112]

--2024-

In [16]:
from utils import load_collection_lookup
from data import InferenceRankingIterator, InferenceDataset
from collator import RankingCollator
import torch
from tqdm import tqdm
# i am using the dev dataset bc I do not have eval dataset for this example

# load the collection lookup
collection = load_collection_lookup("collection_with_texts.jsonl")

eval_ds = InferenceDataset("BM25_E9B2.jsonl",
                            collection,
                            tokenizer,
                            at=100, #max docs pq q
                            iterator_class=InferenceRankingIterator)

dataloader = torch.utils.data.DataLoader(eval_ds,
                                         batch_size=32,
                                         pin_memory=True,
                                         collate_fn=RankingCollator(tokenizer))

In [17]:
_sample = next(iter(dataloader))

_sample["inputs"].input_ids.shape

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([32, 469])

In [18]:
from collections import defaultdict

run_dict = defaultdict(list) # q_id:[{"doc_id": doc_id, "score": prob}]

for sample in tqdm(dataloader):
  _inputs = sample["inputs"].to("cuda") # send inputs to gpu

  # run inference
  with torch.no_grad():
    logits = model(**_inputs).logits # 0->value, 1->value

    # based on the logits, we normalize to have a probabilistic distribution
    # and we extract the probability of that query-doc pair being relevant!
    prob_relevant_pair = torch.nn.functional.softmax(logits, dim=-1)[:,1] # [0-1]
    # 32,

  for i, q_id in enumerate(sample["id"]):
    run_dict[q_id].append({"doc_id":sample["doc_id"][i],
                           "score":prob_relevant_pair[i].item()})


100%|██████████| 313/313 [04:06<00:00,  1.27it/s]


In [19]:
# lest sort by the neural IR model prob and we have our reranked order!
for q_id in run_dict:
  run_dict[q_id].sort(key=lambda x:-x["score"]) #prob


In [21]:
run_dict

defaultdict(list,
            {'6031270b1cb411341a00012c': [{'doc_id': '27822311',
               'score': 0.9999489784240723},
              {'doc_id': '26333682', 'score': 0.9999419450759888},
              {'doc_id': '29321361', 'score': 0.9999058246612549},
              {'doc_id': '27525637', 'score': 0.9998927116394043},
              {'doc_id': '26279649', 'score': 0.9998822212219238},
              {'doc_id': '29177010', 'score': 0.999880313873291},
              {'doc_id': '11694223', 'score': 0.9998773336410522},
              {'doc_id': '2606473', 'score': 0.9998733997344971},
              {'doc_id': '8660052', 'score': 0.9998605251312256},
              {'doc_id': '8817327', 'score': 0.9997597336769104},
              {'doc_id': '15533383', 'score': 0.9996532201766968},
              {'doc_id': '32924381', 'score': 0.999477207660675},
              {'doc_id': '15127762', 'score': 0.9841682314872742},
              {'doc_id': '26273451', 'score': 0.9618173241615295},
      